In [1]:
# imports
import requests
import os

import pandas as pd


from datetime import timedelta
from icalendar import Calendar, Event, Alarm
import pytz

In [2]:
# configuration parameters

# date range
year = 2024
month = 6

# location
city = "London"
country = "United Kingdom"

# calculation settings (api settings)

# calculation method
# 0 - Shia Ithna-Ashari
# 1 - University of Islamic Sciences, Karachi
# 2 - Islamic Society of North America
# 3 - Muslim World League
# 4 - Umm Al-Qura University, Makkah
# 5 - Egyptian General Authority of Survey
# 7 - Institute of Geophysics, University of Tehran
# 8 - Gulf Region
# 9 - Kuwait
# 10 - Qatar
# 11 - Majlis Ugama Islam Singapura, Singapore
# 12 - Union Organization islamic de France
# 13 - Diyanet İşleri Başkanlığı, Turkey
# 14 - Spiritual Administration of Muslims of Russia
# 15 - Moonsighting Committee Worldwide (also requires shafaq parameter)
# 16 - Dubai (unofficial)
# 99 - Custom. See https://aladhan.com/calculation-methods
method = 2

# school
# 0 for Shafi (or the standard way), 1 for Hanafi.
school = 1


# output preferences (how many output files, rounding options)
slot_duration = 15
ceil_to = "30T"
num_timings = 1
which_prayers = ["Dhuhr", "Asr", "Maghrib"]
# which_prayers = ["Fajr", "Dhuhr", "Asr", "Maghrib", "Isha"]



In [3]:
# forklift
# alfred
# copy'em
# bartender
# guidance
# notion calendar
# warp
# cursor ai
# arc browser (invite only)


In [4]:
def prepare_api_call_url() -> str:

    """
    https://aladhan.com/prayer-times-api#GetCalendarByCitys
    """
    
    api_call_str = f"http://api.aladhan.com/v1/calendarByCity/" \
    f"{year}/{month}?city={city}&country={country}" \
    f"&method={method}&school={school}"

    api_call_str = api_call_str.replace(" ", "%20")

    return api_call_str

api_call_str = prepare_api_call_url()

print(api_call_str)



http://api.aladhan.com/v1/calendarByCity/2024/6?city=London&country=United%20Kingdom&method=2&school=1


In [5]:
def get_api_data(api_call_str: str) -> dict:
    response = requests.get(api_call_str)
    raw_output = response.json()['data']
    return raw_output

raw_output = get_api_data(api_call_str)


In [6]:
df_raw = pd.DataFrame(raw_output)
df = df_raw.copy()
df.head()



,timings,date,meta
0,"{'Fajr': '02:54 (BST)', 'Sunrise': '04:49 (BST...","{'readable': '01 Jun 2024', 'timestamp': '1717...","{'latitude': 51.509648, 'longitude': -0.099076..."
1,"{'Fajr': '02:53 (BST)', 'Sunrise': '04:48 (BST...","{'readable': '02 Jun 2024', 'timestamp': '1717...","{'latitude': 51.509648, 'longitude': -0.099076..."
2,"{'Fajr': '02:53 (BST)', 'Sunrise': '04:47 (BST...","{'readable': '03 Jun 2024', 'timestamp': '1717...","{'latitude': 51.509648, 'longitude': -0.099076..."
3,"{'Fajr': '02:53 (BST)', 'Sunrise': '04:46 (BST...","{'readable': '04 Jun 2024', 'timestamp': '1717...","{'latitude': 51.509648, 'longitude': -0.099076..."
4,"{'Fajr': '02:53 (BST)', 'Sunrise': '04:46 (BST...","{'readable': '05 Jun 2024', 'timestamp': '1717...","{'latitude': 51.509648, 'longitude': -0.099076..."


In [7]:
def preprocess_api_data(df):
    df_prayer_times = pd.DataFrame(pd.to_datetime(df["date"].apply(pd.Series)["gregorian"].apply(pd.Series)["date"], format= '%d-%m-%Y').dt.date, columns=["date"])

    df_temp = df["timings"].apply(pd.Series)[which_prayers].copy()
    for prayer in which_prayers:
        df_prayer_times[f"{prayer}"] = pd.to_datetime(df_temp[prayer].str[:5], format= '%H:%M')

    # round to nearest 15 minutes
    for prayer in which_prayers:
        df_prayer_times[f"{prayer}_ceil"] = df_prayer_times[prayer].apply(lambda x: x.ceil(ceil_to))

    return df_prayer_times

df_prayer_times = preprocess_api_data(df)
df_prayer_times.head()



/var/folders/l2/2y9xwgnx5nbgs7lhwg9xyq8m0000gn/T/ipykernel_36003/2562876756.py:10: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_prayer_times[f"{prayer}_ceil"] = df_prayer_times[prayer].apply(lambda x: x.ceil(ceil_to))
/var/folders/l2/2y9xwgnx5nbgs7lhwg9xyq8m0000gn/T/ipykernel_36003/2562876756.py:10: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_prayer_times[f"{prayer}_ceil"] = df_prayer_times[prayer].apply(lambda x: x.ceil(ceil_to))
/var/folders/l2/2y9xwgnx5nbgs7lhwg9xyq8m0000gn/T/ipykernel_36003/2562876756.py:10: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_prayer_times[f"{prayer}_ceil"] = df_prayer_times[prayer].apply(lambda x: x.ceil(ceil_to))


,date,Dhuhr,Asr,Maghrib,Dhuhr_ceil,Asr_ceil,Maghrib_ceil
0,2024-06-01,1900-01-01 12:58:00,1900-01-01 18:31:00,1900-01-01 21:09:00,1900-01-01 13:00:00,1900-01-01 19:00:00,1900-01-01 21:30:00
1,2024-06-02,1900-01-01 12:58:00,1900-01-01 18:32:00,1900-01-01 21:10:00,1900-01-01 13:00:00,1900-01-01 19:00:00,1900-01-01 21:30:00
2,2024-06-03,1900-01-01 12:59:00,1900-01-01 18:32:00,1900-01-01 21:11:00,1900-01-01 13:00:00,1900-01-01 19:00:00,1900-01-01 21:30:00
3,2024-06-04,1900-01-01 12:59:00,1900-01-01 18:33:00,1900-01-01 21:12:00,1900-01-01 13:00:00,1900-01-01 19:00:00,1900-01-01 21:30:00
4,2024-06-05,1900-01-01 12:59:00,1900-01-01 18:33:00,1900-01-01 21:13:00,1900-01-01 13:00:00,1900-01-01 19:00:00,1900-01-01 21:30:00


In [8]:
def calculate_ical_df(df_prayer_times, which_prayers):
    dates_for_each_time = []
    for prayer in which_prayers:
        df_temp = df_prayer_times.groupby(f"{prayer}_ceil").agg({"date":["min", "max"]}).reset_index()
        df_temp.columns = ["prayer_time", "first_date", "last_date"]
        df_temp["prayer"] = prayer

        df_temp["first_date"] = pd.to_datetime(df_temp["first_date"]) + pd.to_timedelta(df_temp["prayer_time"].dt.strftime('%H:%M:%S'))
        df_temp["last_date"] = pd.to_datetime(df_temp["last_date"]) + pd.to_timedelta(df_temp["prayer_time"].dt.strftime('%H:%M:%S'))

        dates_for_each_time.append(df_temp)

    df_dates_for_each_time = pd.concat(dates_for_each_time)
    return df_dates_for_each_time
df_dates_for_each_time = calculate_ical_df(df_prayer_times, which_prayers)
df_dates_for_each_time.head()



,prayer_time,first_date,last_date,prayer
0,1900-01-01 13:00:00,2024-06-01 13:00:00,2024-06-12 13:00:00,Dhuhr
1,1900-01-01 13:30:00,2024-06-13 13:30:00,2024-06-30 13:30:00,Dhuhr
0,1900-01-01 19:00:00,2024-06-01 19:00:00,2024-06-30 19:00:00,Asr
0,1900-01-01 21:30:00,2024-06-01 21:30:00,2024-06-30 21:30:00,Maghrib


In [9]:
def create_ics_text_from_definition(prayer, start_date, duration_minutes, last_date, alarm_minutes):

    # start_date = datetime(2024, 5, 1, 16, 0, 0)  # Start at 16:00
    # last_date = datetime(2024, 5, 31)

    start_date = pd.to_datetime(start_date)
    start_date = pytz.timezone('Europe/London').localize(start_date)
    duration = timedelta(minutes=duration_minutes)
    alarm_minutes = timedelta(minutes=-alarm_minutes)

    # Initialize Calendar and Event
    cal = Calendar()

    cal.add('prodid', '-//My calendar builder//asa.asa//')
    cal.add('version', '2.0')

    event = Event()

    # Add event properties
    event.add('summary', prayer)
    event.add('dtstart', start_date)
    event.add('dtend', start_date + duration)
    event.add('rrule', {
        'freq': 'weekly',
        'interval': 1,
        'byday': ['MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU'],  # Corrected byday parameter
        'until': last_date
    })

    # Add reminder
    alarm = Alarm()
    alarm.add('action', 'DISPLAY')
    alarm.add('description', f'Reminder: {prayer} in {alarm_minutes} minutes')
    alarm.add('trigger', alarm_minutes)
    event.add_component(alarm)

    # Add event to the calendar
    cal.add_component(event)

    # Output the calendar
    # print(cal.to_ical().decode('utf-8'))

    return cal


list_of_ics_per_prayer = {}
for prayer in which_prayers:
    list_of_ics_per_prayer[prayer] = []
    prayer_times = df_dates_for_each_time.loc[df_dates_for_each_time["prayer"] == prayer]
    for idx, line in prayer_times.iterrows():
        list_of_ics_per_prayer[prayer].append(create_ics_text_from_definition(
            line["prayer"], line["first_date"], slot_duration, line["last_date"], 10))

print(list_of_ics_per_prayer["Dhuhr"][0])

VCALENDAR({'PRODID': vText('b'-//My calendar builder//asa.asa//''), 'VERSION': vText('b'2.0'')}, VEVENT({'SUMMARY': vText('b'Dhuhr''), 'DTSTART': vDDDTypes(2024-06-01 13:00:00+01:00, Parameters({'TZID': 'Europe/London'})), 'DTEND': vDDDTypes(2024-06-01 13:15:00+01:00, Parameters({'TZID': 'Europe/London'})), 'RRULE': vRecur({'FREQ': 'weekly', 'INTERVAL': 1, 'BYDAY': ['MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU'], 'UNTIL': Timestamp('2024-06-12 13:00:00')})}, VALARM({'ACTION': vText('b'DISPLAY''), 'DESCRIPTION': vText('b'Reminder: Dhuhr in -1 day\\, 23:50:00 minutes''), 'TRIGGER': vDDDTypes(-1 day, 23:50:00, Parameters({}))})))


In [11]:
# Save ics files
for prayer in which_prayers:
    for ii, cal in enumerate(list_of_ics_per_prayer[prayer]):
        with open(os.path.join(os.getcwd(), 'output', f'{prayer}_{ii}.ics'), 'wb') as f:
            f.write(cal.to_ical())

In [13]:
list(range(2024+1, 2024 - 2, -1))

[2025, 2024, 2023]